In [1]:
# extract different view of data
# view two, node2vec
setting = "n2v"
Dataset = "pubmed"
viewTwoFilesDir = "../Data/"+Dataset+"/vectors/"+setting+"/data=Meta-alg=N2V-l2=1.0-n2v_p=0.85-iteration=100-no_self_predict=1-idx=0.emb"
viewTwoVectors = []

with open(viewTwoFilesDir, 'r', encoding = 'utf8') as f:
    for line in f:
        read_data = line.split(" ")
        if(len(read_data[0])<=8):
            paper_Vectors = read_data
            viewTwoVectors.append(paper_Vectors)
f.close()
viewTwoVectors = viewTwoVectors[1:]
print("Total vector records:",len(viewTwoVectors))
print(viewTwoVectors[0])

Total vector records: 8602916
['22516865', '0.0109272', '0.126011', '0.186979', '0.0496719', '0.0373553', '0.0458918', '-0.119893', '0.217118', '0.0524591', '0.237477', '0.191269', '-0.0277055', '0.0290957', '-0.0366833', '0.118964', '0.0654807', '-0.0335345', '-0.0900123', '0.128621', '0.0561669', '-0.087823', '-0.0882296', '0.0740289', '0.082104', '0.0269581', '-0.0346502', '0.0153376', '0.104666', '0.0908716', '-0.085694', '-0.111344', '0.0787209', '-0.17003', '-0.103366', '-0.0832094', '-0.210496', '0.153037', '-0.0342884', '0.0698413', '-0.0719641', '-0.0535707', '0.172399', '0.106226', '-0.0593672', '-0.0348048', '-0.0863189', '-0.0801566', '-0.0665761', '0.0673258', '0.0306541', '-0.0896316', '-0.00800971', '-0.174798', '-0.0252528', '0.0098563', '0.0230368', '0.0282268', '-0.0366493', '-0.131323', '0.0318188', '-0.00778704', '-0.0608064', '-0.0860078', '0.215632', '0.0209927', '-0.0953191', '-0.191736', '-0.0741615', '0.151972', '-0.0522046', '-0.11081', '0.134878', '0.090797',

In [2]:
import re
import os

# collect pid
fileDir = "../Data/"+Dataset+"/filteredSameNameAuthor/filter=10/"
fileList = os.listdir(fileDir)
fileList.sort()
print(fileList)

['alfredo martinez.txt', 'alfredo martinez0.txt', 'alfredo martinez1.txt', 'amit patel.txt', 'amit patel0.txt', 'amit patel1.txt', 'ana castro.txt', 'ana castro0.txt', 'ana castro1.txt', 'ana castro2.txt', 'anna ferrari.txt', 'anna ferrari0.txt', 'anna ferrari1.txt', 'bin liu.txt', 'bin liu0.txt', 'bin liu1.txt', 'carmen moreno.txt', 'carmen moreno0.txt', 'carmen moreno1.txt', 'carmen torres.txt', 'carmen torres0.txt', 'carmen torres1.txt', 'chao liu.txt', 'chao liu0.txt', 'chao liu1.txt', 'cheng luo.txt', 'cheng luo0.txt', 'cheng luo1.txt', 'chung-may yang.txt', 'chung-may yang0.txt', 'chung-may yang1.txt', 'david g lloyd.txt', 'david g lloyd0.txt', 'david g lloyd1.txt', 'fang liu.txt', 'fang liu0.txt', 'fang liu1.txt', 'feng liu.txt', 'feng liu0.txt', 'feng liu1.txt', 'feng xu.txt', 'feng xu0.txt', 'feng xu1.txt', 'francisco esteves.txt', 'francisco esteves0.txt', 'francisco esteves1.txt', 'francisco j blanco.txt', 'francisco j blanco0.txt', 'francisco j blanco1.txt', 'giovanni volpe

In [3]:
# remove author(positive sample) from other(negative sample)
import random
def extractNegativeSample(positiveSample, allSample):
    negativeSample = [x for x in allSample if x not in positiveSample]
    print("Total negative sample size:", len(negativeSample))
    return negativeSample

In [11]:
# collect class vectors
import pandas as pd
import numpy as np

def extractVectors(authors_pids, allPaperVectors):
    appended_data = []
    for label, author in enumerate(authors_pids):
        data = []
        labels = []
        for paper_Vectors in allPaperVectors:
            if paper_Vectors[0] in author:
                data.append(paper_Vectors)
                labels.append(label)
        print("Class ",label," sample size: ", len(data))
        # create df save one author data 
        authordf = pd.DataFrame(data)
        authordf['label'] = labels
        appended_data.append(authordf)
    # add all together
    dataset = pd.concat(appended_data, axis=0,ignore_index=True)
    # shuffle it
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    # print shape for confirmation
    print(dataset.shape)
    return dataset

In [17]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import (precision_score, recall_score,f1_score,accuracy_score)
# cross validation
def k_fold_cv(data, label, paperID, classifier, k=10):
    kf = KFold(n_splits=k, shuffle=False)
    allTrueLabel = []
    allPredLabel = []
    for counter,(train_index, test_index) in enumerate(kf.split(data)):
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # split train and test
        data_train, data_test = data.iloc[train_index], data.iloc[test_index]
        label_train, test_true_label = label.iloc[train_index], label.iloc[test_index]
        # fit data to svm
        classifier.fit(data_train, label_train)
        # get predicted label
        label_pred = classifier.predict(data_test)
        allTrueLabel.extend(test_true_label)
        allPredLabel.extend(label_pred)
#         # get predict proba
#         proba = classifier.predict_proba(data_test)
        # find out which sample cause the issue
        print("Pred: ",label_pred)
        print("True: ", test_true_label.values.tolist())
        print("Mislabeled sample: ",end='')
        for i in range(len(test_true_label)):
            if(label_pred[i]!=test_true_label[test_index[i]]):
                print(paperID[test_index[i]]+",",end='')
        print()
        # print("True positive: {tp}, False positive: {fp}, False negative: {fn}, True negative: {tn}"
        # .format(tp=round_tp, fp=round_fp, fn=round_fn, tn=round_tn))

    accuracy = accuracy_score(allTrueLabel, allPredLabel)
    f1 = f1_score(allTrueLabel, allPredLabel,average='binary')
    precision = precision_score(allTrueLabel, allPredLabel)
    recall = recall_score(allTrueLabel, allPredLabel)
    tn,fp,fn,tp = metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel()
    
    print("Classifier: ",classifier)
    print(metrics.classification_report(allTrueLabel, allPredLabel))
    print(metrics.confusion_matrix(allTrueLabel, allPredLabel).ravel())
    print("Accuracy: ",accuracy)
    print("F1: ", f1)
    print("Precision: ", precision)
    print("Recall: ", recall)
    
    return accuracy, f1, precision, recall, tn, fp, fn, tp
    # return ppv, npv, specificity, sensitivity, accuracy, f1proba = linear_svc.predict_proba(allDatas)

In [7]:
# hard code to read the file one by one
# store the features for classification
author_pids = []
other_pids = []
# author as positive sample, other as all samples
with open(fileDir+"kyung su kim1.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        author_pids.extend(line.strip().split(" "))

with open(fileDir+"kyung su kim.txt", 'r', encoding = 'utf8') as f:
    for line in f:
        other_pids.extend(line.strip().split(" "))
        
print(author_pids)
print(other_pids[0])

['24856736', '24553547', '23704754', '24981010', '24495814', '23632268', '22664745', '23648213', '23932734', '23603152', '23648567', '22867838', '22975022', '22867824', '23380094', '22683079', '23187987', '22487663', '22748697', '22980485', '21824465', '22227383', '22391544', '22463971', '21803673', '22533576', '22424993', '22657731', '22932993', '22322281', '20542398', '22372785', '21633052', '21908139', '21129911', '21610354', '20511643', '18996670', '21236547', '18722743', '19913985', '20599213', '20154545', '20837252', '19336669', '19542400', '19111959', '18638308', '19119441', '18022782', '18756061', '18821148', '18370984', '18657674', '17590258']
23528677


In [13]:
# extract negative Sample
NegativeSample_pid = extractNegativeSample(author_pids, other_pids)
print("Choicen negative sample ", len(NegativeSample_pid))
all_author = []
all_author.append(author_pids)
all_author.append(NegativeSample_pid)

Total negative sample size: 14
Choicen negative sample  14


In [14]:
data = extractVectors(all_author,viewTwoVectors)
print(data.shape)

Class  0  sample size:  55
Class  1  sample size:  14
(69, 102)
(69, 102)


In [15]:
label = data['label']
pid = data[0]
data = data.drop([0,'label'], axis=1)
print(data.shape)

(69, 100)


In [18]:
from sklearn import svm
linear_svc = svm.SVC(kernel='linear', class_weight='balanced', probability=True,cache_size=4000)
accuracy, f1, precision, recall, tn, fp, fn, tp= k_fold_cv(data, label, pid, linear_svc,10)


Pred:  [1 0 0 1 0 0 0]
True:  [0, 0, 0, 0, 0, 0, 0]
Mislabeled sample: 24553547,21236547,
Pred:  [1 0 1 0 0 0 0]
True:  [1, 1, 0, 0, 0, 0, 0]
Mislabeled sample: 20825813,23648567,
Pred:  [1 0 0 0 1 0 0]
True:  [0, 0, 1, 0, 0, 1, 0]
Mislabeled sample: 18821148,19097734,23380094,18660392,
Pred:  [0 0 1 0 0 0 0]
True:  [0, 0, 0, 0, 0, 0, 0]
Mislabeled sample: 22463971,
Pred:  [0 0 0 1 1 0 0]
True:  [1, 0, 1, 0, 0, 0, 0]
Mislabeled sample: 22101201,20825912,23932734,22980485,
Pred:  [0 0 0 1 1 1 1]
True:  [0, 0, 0, 0, 0, 0, 1]
Mislabeled sample: 18638308,22683079,18756061,
Pred:  [0 0 0 0 1 0 0]
True:  [0, 0, 0, 0, 1, 1, 0]
Mislabeled sample: 21982990,
Pred:  [1 0 1 1 0 1 0]
True:  [0, 0, 1, 0, 1, 0, 0]
Mislabeled sample: 20511643,18657674,21722666,19913985,
Pred:  [0 0 0 0 1 0 1]
True:  [0, 0, 0, 0, 1, 1, 0]
Mislabeled sample: 19996791,21908139,
Pred:  [1 1 0 1 1 0]
True:  [0, 0, 0, 0, 1, 0]
Mislabeled sample: 22664745,21803673,21610354,
Classifier:  SVC(C=1.0, cache_size=4000, class_weig